Let's solve a rubik's cube.

In [107]:
import numpy as np
import torch
import matplotlib.pyplot as plt

class Cube:
    def __init__(self):
        self.cube = np.zeros((5,5,5))
        self.cube[0,1:4,1:4] += 2
        self.cube[-1,1:4,1:4] += 1
        self.cube[1:4,1:4,0] += 3
        self.cube[1:4,1:4,-1] += 4
        self.cube[1:4,0,1:4] += 5
        self.cube[1:4,-1,1:4] += 6

    @property
    def f(self):
        # front clockwise
        self.cube[-2] = np.flip(self.cube[-2].T,1)
        self.cube[-1] = np.flip(self.cube[-1].T,1)
        return self

    @property
    def F(self):
        # front counter-clockwise
        self.cube[-2] = np.flip(self.cube[-2].T,0)
        self.cube[-1] = np.flip(self.cube[-1].T,0)
        return self

    @property
    def s(self):
        # mid x-y clockwise
        self.cube[2] = np.flip(self.cube[2].T,0)
        return self

    @property
    def S(self):
        # mid x-y counter-clockwise
        self.cube[2] = np.flip(self.cube[2].T,1)
        return self

    @property
    def b(self):
        # back clockwise
        self.cube[1] = np.flip(self.cube[1].T,0)
        self.cube[0] = np.flip(self.cube[0].T,0)
        return self

    @property
    def B(self):
        # back counter-clockwise
        self.cube[1] = np.flip(self.cube[1].T,1)
        self.cube[0] = np.flip(self.cube[0].T,1)
        return self

    @property
    def l(self):
        # left clockwise
        self.cube[:,:,:2] = np.rot90(self.cube[:,:,:2],axes=(0,1))
        return self

    @property
    def L(self):
        # left counter-clockwise
        self.cube[:,:,:2] = np.rot90(self.cube[:,:,:2],axes=(1,0))
        return self

    @property
    def m(self):
        # mix y-z clockwise
        self.cube[:,:,2] = np.rot90(self.cube[:,:,2],axes=(0,1))
        return self

    @property
    def M(self):
        # mix y-z counter-clockwise
        self.cube[:,:,2] = np.rot90(self.cube[:,:,2],axes=(1,0))
        return self

    @property
    def r(self):
        # right clockwise
        self.cube[:,:,3:] = np.rot90(self.cube[:,:,3:],axes=(0,1))
        return self

    @property
    def R(self):
        # right counter-clockwise
        self.cube[:,:,3:] = np.rot90(self.cube[:,:,3:],axes=(1,0))
        return self

    @property
    def u(self):
        # up clockwise
        self.cube[:,1] = np.flip(self.cube[:,1].T,-1)
        self.cube[:,0] = np.flip(self.cube[:,0].T,-1)
        return self

    @property
    def U(self):
        # up counter-clockwise
        self.cube[:,1] = np.flip(self.cube[:,1].T,0)
        self.cube[:,0] = np.flip(self.cube[:,0].T,0)
        return self

    @property
    def e(self):
        # mid x-z clockwise
        self.cube[:,2] = np.flip(self.cube[:,2].T,-1)
        return self

    @property
    def E(self):
        # mid x-z counter-clockwise
        self.cube[:,2] = np.flip(self.cube[:,2].T,0)
        return self

    @property
    def d(self):
        # down clockwise
        self.cube[:,-2] = np.flip(self.cube[:,-2].T,0)
        self.cube[:,-1] = np.flip(self.cube[:,-1].T,0)
        return self

    @property
    def D(self):
        # down counter-clockwise
        self.cube[:,-2] = np.flip(self.cube[:,-2].T,-1)
        self.cube[:,-1] = np.flip(self.cube[:,-1].T,-1)
        return self




In [106]:
c = Cube()
c.E.cube

array([[[0., 0., 0., 0., 0.],
        [0., 2., 2., 2., 0.],
        [0., 4., 4., 4., 0.],
        [0., 2., 2., 2., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 5., 5., 5., 0.],
        [3., 0., 0., 0., 4.],
        [2., 0., 0., 0., 1.],
        [3., 0., 0., 0., 4.],
        [0., 6., 6., 6., 0.]],

       [[0., 5., 5., 5., 0.],
        [3., 0., 0., 0., 4.],
        [2., 0., 0., 0., 1.],
        [3., 0., 0., 0., 4.],
        [0., 6., 6., 6., 0.]],

       [[0., 5., 5., 5., 0.],
        [3., 0., 0., 0., 4.],
        [2., 0., 0., 0., 1.],
        [3., 0., 0., 0., 4.],
        [0., 6., 6., 6., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 3., 3., 3., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]]])